# Importing and reading in data

In addition to our normal pandas and numpy, I've chosen to import some packages associated with web scraping and preprocessing, things that will be utilized later on in the project.

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import os
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor

In [ ]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Preprocessing player data

In [ ]:
# Creating list of team names for looping through function
team_names = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL','DET', 'HOU', 'KC', 'LAA', 'LAD', 'MIA', 
              'MIL', 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SEA', 'SF', 'STL', 'TB', 'TEX', 'TOR', 'WAS']

In [ ]:
def df_preprocessing(team_name) :
    
    batting_path = os.path.join('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/team-player-data/Offense', f'{team_name}batting.csv')
    pitching_path = os.path.join('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/team-player-data/Pitching', f'{team_name}pitching.csv')
    roster_path = os.path.join('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/team-player-data/Current Rosters', f'{team_name}roster.csv')
    
    batting_df = pd.read_csv(batting_path)
    pitching_df = pd.read_csv(pitching_path)
    roster_df = pd.read_csv(roster_path) 
    
# Roster preprocessing
    roster_df = roster_df.drop(columns = ['Rk', 'Uni', 'Unnamed: 3', 'Unnamed: 4', 'OnActv', 'IL', 'DoB', '1stYr', 'Name-additional'])
    removable_substrings = ['*', '#', '?', ' (15-day IL)', ' (7-day IL)', ' (10-day IL)', ' (60-day IL)', ' (40-man)']
    name_pattern = '|'.join(map(re.escape, removable_substrings))
    roster_df['Name'] = roster_df['Name'].str.replace(name_pattern, '')
    roster_df.dropna(how = 'all', inplace = True)
    excluded_columns_roster = ['Name', 'B', 'T', 'Ht']
    for column in roster_df.columns :
        if column not in excluded_columns_roster :
            roster_df[column] = roster_df[column].astype(int)

# Batting preprocessing
    batting_df = batting_df.rename(columns = {'Pos' : 'Position'}).drop(columns = ['Rk', 'Name-additional'])
    removable_substrings = ['*', '#', '?', ' (15-day IL)', ' (10-day IL)', ' (60-day IL)', ' (40-man)', ' (7-day IL)', ' (DFA)']
    name_pattern = '|'.join(map(re.escape, removable_substrings))
    batting_df['Name'] = batting_df['Name'].str.replace(name_pattern, '')
    batting_df.dropna(how = 'all', inplace = True)
    excluded_columns_bat = ['Position', 'Name', 'BA', 'OBP', 'SLG', 'OPS']
    for column in batting_df.columns :
        if column not in excluded_columns_bat :
            batting_df[column] = pd.to_numeric(batting_df[column], errors = 'coerce')
            batting_df[column].fillna(0, inplace = True)
            batting_df[column] = batting_df[column].astype(int)
    batting_df = batting_df.drop(batting_df[batting_df['Position'] == 'P'].index)
    batting_df.insert(1, 'Team', team_name)

# Pitching preprocessing
    pitching_df = pitching_df.rename(columns = {'Pos' : 'Position'}).drop(columns = ['Rk', 'Name-additional'])
    removable_substrings = ['*', '#', '?', ' (15-day IL)', ' (7-day IL)', ' (10-day IL)', ' (60-day IL)', ' (40-man)', ' (DFA)']
    name_pattern = '|'.join(map(re.escape, removable_substrings))
    pitching_df['Name'] = pitching_df['Name'].str.replace(name_pattern, '')
    pitching_df.dropna(how = 'all', inplace = True)
    pitching_df.dropna(subset = ['W-L%', 'SO/W'], inplace = True)
    pitching_df['Position'].fillna('P', inplace = True)
    excluded_columns_pitch = ['Position', 'Name', 'IP', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9']
    for column in pitching_df.columns :
        if column not in excluded_columns_pitch :
            pitching_df[column] = pitching_df[column].astype(int)
    pitching_df.insert(1, 'Team', team_name)
    
    batting_df[batting_df['Name'].isin(roster_df['Name'])].reset_index(drop = True, inplace = True)
    pitching_df[pitching_df['Name'].isin(roster_df['Name'])].reset_index(drop = True, inplace = True)
    
# Saving updated dataframes

    filtered_batting_path = os.path.join('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/team-player-data/Filtered_Offense', f'{team_name}batting_filtered.csv')
    filtered_pitching_path = os.path.join('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/team-player-data/Filtered_Pitching', f'{team_name}pitching_filtered.csv')
    batting_df.to_csv(filtered_batting_path, index = False)
    pitching_df.to_csv(filtered_pitching_path, index = False)

In [ ]:
if __name__ == '__main__' :
    num_threads = 30

    with ThreadPoolExecutor(max_workers = num_threads) as executor :
# Use the map function to apply the df_preprocessing function to each team concurrently
        try :
            executor.map(df_preprocessing, team_names)
        except Exception as e :
            print(f"Error occurred: {e}")

# Merging the filtered data for each team into a single DataFrame if needed
    batting_dfs = []
    for team_name in team_names:
        filtered_batting_path = f'/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/team-player-data/Filtered_Offense/{team_name}batting_filtered.csv'
        df = pd.read_csv(filtered_batting_path)
        batting_dfs.append(df)
    batting_df = pd.concat(batting_dfs)
    
    pitching_dfs = []
    for team_name in team_names:
        filtered_pitching_path = f'/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/team-player-data/Filtered_Pitching/{team_name}pitching_filtered.csv'
        df = pd.read_csv(filtered_pitching_path)
        pitching_dfs.append(df)
    pitching_df = pd.concat(pitching_dfs)

In [ ]:
# Quick reset of indices for concatenated dataframes
batting_df.reset_index(drop = True, inplace = True)
pitching_df.reset_index(drop = True, inplace = True)

In [ ]:
batting_df

In [ ]:
pitching_df

# Year-by-Year MLB Team Data

In this section I will web scrape for team data year-by-year, such that drafting players can be compared against what statistics winning teams display. A side note on WAR: the average war for a World Series-winning team, is 47.0, with an average WAR for batting players at 29.7 and pitchers 17.3. 

Source: https://sabr.org/journal/article/war-and-the-world-series-is-war-an-indicator-of-october-success/

In [ ]:
wswinners = pd.read_csv('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/WSWinners.csv')

In [ ]:
wswinners

In [ ]:
wswinners = wswinners.drop(columns = ['Series MVP'])
wswinners.head(30)

In [ ]:
wswinners = wswinners.drop(28)
wswinners.head(30)

In [ ]:
wswinners.dtypes

In [ ]:
wswinners['Year'] = wswinners['Year'].astype(int)

In [ ]:
real_winners = []
for index, row in wswinners.iterrows():
    if row['Wins'] > row['Wins.1']:
        real_winners.append(row['AL Winner'])
    else:
        real_winners.append(row['NL Winner'])

In [ ]:
real_winners

In [ ]:
wswinners['World Series Champions'] = real_winners
wswinners = wswinners.drop(columns = ['AL Winner', 'Wins', 'Wins.1', 'NL Winner'])
wswinners

In [ ]:
# have to make slightly different team names list because baseball reference does them differently
bref_team_names = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL','DET', 'HOU', 'KCR', 'ANA', 'LAD', 'FLA', 
              'MIL', 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SDP', 'SEA', 'SFG', 'STL', 'TBD', 'TEX', 'TOR', 'WSN']

In [ ]:
team_dataframes = []
for team_abbr in bref_team_names:
    url = f'https://www.baseball-reference.com/teams/{team_abbr}/batteam.shtml#all_yby_team_bat'
    
    response = requests.get(url)
    
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the "Year-by-Year Team Batting" table by its id
        tables = soup.find_all('table')

        if tables:
            # Convert the first table data to a DataFrame
            team_df = pd.read_html(str(tables[0]))[0]

            # Add a "Team" column to store the team's abbreviation
            team_df['Team'] = team_abbr

            # Append the DataFrame to the list
            team_dataframes.append(team_df)


        else:
            print(f"No tables found on the page for {team_abbr}")
    else:
        print(f"Failed to retrieve data for {team_abbr}")
        
# Concatenate all DataFrames into a single DataFrame
all_team_data = pd.concat(team_dataframes, ignore_index=True)

# Save the combined data to a CSV file (optional)
all_team_data.to_csv('team_batting_data.csv', index=False)

# Display the first few rows of the combined DataFrame
print(all_team_data.head())

In [ ]:
all_team_data_batting = all_team_data
all_team_data_batting

In [ ]:
team_dataframes = []
for team_abbr in bref_team_names:
    url = f'https://www.baseball-reference.com/teams/{team_abbr}/pitchteam.shtml#all_yby_team_pitch'
    
    response = requests.get(url)
    
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the "Year-by-Year Team Batting" table by its id
        tables = soup.find_all('table')

        if tables:
            # Convert the first table data to a DataFrame
            team_df = pd.read_html(str(tables[0]))[0]

            # Add a "Team" column to store the team's abbreviation
            team_df['Team'] = team_abbr

            # Append the DataFrame to the list
            team_dataframes.append(team_df)


        else:
            print(f"No tables found on the page for {team_abbr}")
    else:
        print(f"Failed to retrieve data for {team_abbr}")
        
# Concatenate all DataFrames into a single DataFrame
all_team_data_pitching = pd.concat(team_dataframes, ignore_index=True)

# Save the combined data to a CSV file (optional)
all_team_data_pitching.to_csv('team_pitching_data.csv', index=False)

# Display the first few rows of the combined DataFrame
print(all_team_data_pitching.head())

In [ ]:
all_team_data_pitching

In [ ]:
all_team_data_pitching.to_csv('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/all_team_data_pitching.csv', index = False)

In [ ]:
all_team_data_batting.to_csv('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/all_team_data_batting.csv', index = False)

In [ ]:
name_abbr_dict = {'Arizona Diamondbacks':'ARI', 'Atlanta Braves':'ATL', 'Baltimore Orioles':'BAL', 
                  'Boston Red Sox':'BOS', 'Chicago Cubs':'CHC', 'Chicago White Sox':'CHW', 'Cincinatti Reds':'CIN',
                  'Cleveland Indians': 'CLE', 'Colorado Rockies':'COL', 'Detroit Tigers':'DET', 'Houston Astros':'HOU',
                  'Kansas City Royals':'KCR', 'Anaheim Angels':'ANA', 'Los Angeles Dodgers':'LAD', 
                  'Florida Marlins':'FLA', 'Milwaukee Brewers':'MIL', 'Minnesota Twins':'MIN', 'New York Mets':'NYM',
                  'New York Yankees':'NYY', 'Oakland Athletics':'OAK', 'Philadelphia Phillies':'PHI', 
                  'Pittsburgh Pirates':'PIT', 'San Diego Padres':'SDP', 'Seattle Mariners':'SEA', 
                  'San Francisco Giants':'SFG', 'St. Louis Cardinals':'STL', 'Tampa Bay Rays':'TBD', 'Texas Rangers':'TEX', 
                  'Toronto Blue Jays':'TOR', 'Washington Nationals':'WSN'}

In [ ]:
all_team_data_batting = pd.read_csv('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/all_team_data_batting.csv')

In [ ]:
all_team_data_batting

In [ ]:
wswinners['Team'] = wswinners['World Series Champions'].map(name_abbr_dict)
wswinners

In [ ]:
wsbat = all_team_data_batting.merge(wswinners, on=['Year', 'Team'], how = 'left')
wsbat['World Series Champions'].count()

In [ ]:
wspitch = all_team_data_pitching.merge(wswinners, on=['Year', 'Team'], how = 'left')
wspitch['World Series Champions'].count()

In [ ]:
wspitching = pd.merge(all_team_data_pitching, wswinners, on=['Year', 'Team'], how='inner')

In [ ]:
wspitching

In [ ]:
wschamps = wspitching.pop('World Series Champions')
wspitching.insert(1, wschamps.name, wschamps)
wspitching = wspitching.drop(columns = ['Team', 'Lg', 'W', 'L', 'Finish'])
wspitching = wspitching.sort_values(by = 'Year', ascending = False).reset_index()
wspitching = wspitching.drop(columns = ['index'])
wspitching

In [ ]:
wschamps1 = wsbatting.pop('World Series Champions')
wsbatting.insert(1, wschamps.name, wschamps)
wsbatting = wsbatting.drop(columns = ['Team', 'Lg', 'W', 'L', 'Finish'])
wsbatting = wsbatting.sort_values(by = 'Year', ascending = False).reset_index()
wsbatting = wsbatting.drop(columns = ['index'])
wsbatting

In [ ]:
wsbattingmeans = wsbatting.drop(columns = ['Year', 'G']).mean()
wspitchingmeans = wspitching.drop(columns = ['Year', 'G']).mean()
wsbattingmeans

In [ ]:
wspitchingmeans

## Placeholder -- below testing on refactoring batting and pitching dataframes to check for WS winner

In [ ]:
all_team_data_batting = all_team_data_batting[(all_team_data_batting['Year'] >= 1998) & (all_team_data_batting['Year'] <= 2022)]
all_team_data_pitching = all_team_data_pitching[(all_team_data_pitching['Year'] >= 1998) & (all_team_data_pitching['Year'] <= 2022)]

In [ ]:
all_team_data_batting

In [ ]:
all_team_data_pitching

In [ ]:
batting_df_WS = all_team_data_batting.merge(wswinners, on=['Year', 'Team'], how='left')
batting_df_WS['WS_Winner'] = batting_df_WS['World Series Champions'].apply(lambda x: 'Yes' if pd.notna(x) else 'No')
batting_df_WS.drop(columns=['World Series Champions'], inplace=True)
batting_df_WS = batting_df_WS.drop(columns = ['Lg', 'W', 'L', 'G', 'Finish'])
teamname = batting_df_WS.pop('Team')
batting_df_WS.insert(1, teamname.name, teamname)
batting_df_WS = batting_df_WS[batting_df_WS['Year'] != 2020]
batting_df_WS = batting_df_WS.rename(columns = {'2B':'Doubles', '3B':'Triples'})
batting_df_WS

In [ ]:
pitching_df_WS = all_team_data_pitching.merge(wswinners, on=['Year', 'Team'], how='left')
pitching_df_WS['WS_Winner'] = pitching_df_WS['World Series Champions'].apply(lambda x: 'Yes' if pd.notna(x) else 'No')
pitching_df_WS.drop(columns=['World Series Champions'], inplace=True)
pitching_df_WS = pitching_df_WS.drop(columns = ['Lg', 'W', 'L', 'G', 'Finish'])
teamname = pitching_df_WS.pop('Team')
pitching_df_WS.insert(1, teamname.name, teamname)
pitching_df_WS = pitching_df_WS[pitching_df_WS['Year'] != 2020]
pitching_df_WS

In [ ]:
pitching_df_WS.to_csv('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/team_pitching_df.csv', index = False)
batting_df_WS.to_csv('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/team_batting_df.csv', index = False)

In [ ]:
merged_df = pitching_df_WS.merge(batting_df_WS, on=['Year', 'Team'], how='inner')

In [ ]:
merged_df

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
merged_df = merged_df.drop(columns=['WS_Winner_x', 'E_y', 'DP_y', 'Fld%_y', 'PA', 'AB']).rename(columns = {'WS_Winner_y':'WS_Winner', 'H_x':'Hits_Allowed', 
                                                                       'R_x':'Runs_Allowed', 'HR_x':'HR_Allowed',
                                                                      'BB_x':'Walks_Allowed', 'SO_x':'SO_Pitch',
                                                                      'E_x':'Errors_Committed', 'DP_x':'GDP',
                                                                      'Fld%_x':'Fielding_Percentage', 'R_y': 'Runs_Scored',
                                                                      'H_y':'Hits_For', 'HR_y':'HR_For', 'BB_y':'Walks_For', 
                                                                      'SO_y':'SO_Bat', 'BatAge':'Batting_Age', 'PAge':'Pitching_Age',
                                                                      'RA/G':'Runs_Allowed_Per_Game', 'R/G':'Runs_Per_Game',
                                                                      '2B':'Doubles', '3B':'Triples'})

In [ ]:
merged_df

In [ ]:
pitchage = merged_df.pop('Pitching_Age')
merged_df.insert(36, pitchage.name, pitchage)
merged_df.iloc[:, 2:37] = merged_df.iloc[:, 2:37].astype(float)
merged_df

In [ ]:
merged_df.to_csv('/Users/jantmann17/Desktop/Portfolio-Projects/MLB-Expansion-Team-Project/all_team_data.csv', index = False)

In [ ]:
merged_df.dtypes